In [ ]:
#
# The MIT License (MIT)

# Copyright (c) 2021, NVIDIA CORPORATION

# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
#

**N.B: This preproc-V2 generates interactions and sessions my removing repeated interactions of the same product** 

In [1]:
from IPython.display import display
import gc
import glob
import os
from functools import partial

import cudf
import cupy
import json
import numpy as np
import nvtabular as nvt
import pandas as pd
from tqdm import tqdm
import pickle

pd.options.display.max_columns = None
tqdm.pandas()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Set data and output paths 
DATA_FOLDER = "/workspace/"
FILENAME_PATTERN_BROWSING = 'browsing_train.csv'
FILENAME_PATTERN_SEARCH = 'search_train.csv'
DATA_PATH_BROWSING = os.path.join(DATA_FOLDER, FILENAME_PATTERN_BROWSING)
DATA_PATH_SEARCH = os.path.join(DATA_FOLDER, FILENAME_PATTERN_SEARCH)
OUTPUT_DIR = '/workspace/coveo_task2_v1_phase2/sessions_wo_repetitions'

* The objective of this notebook is to create sequential features for user sessions and generate classification features: 
    - The product id of first AC event 
    - The purchase of this AC (binary label)  
    


* Four independant sections that create different parquet files : 

   - Feature engineering with Pandas to merge browsing, search and test data and create purchase and AC features: <a href ='#pandas_proc'> Section 1 </a> 

   - Preprocess row interactions to encode categoricals and normalize numerical features using NVTabular :  <a href='#row_workflow'> Section 2 </a>
   
   - Group by interactions to create sessions table using NVTabular:  <a href='#session_workflow'> Section 3 </a>
   
   - Duplicate sessions in train and validation data by truncating the sequence of interactions to different number of actions (0, 2, 4, 6, 8, 10) after the AC : <a href='#session_duplicate'> Section 4 </a>

<h1> <center> <a id='pandas_proc'> Section 1 : Create event table </a></center></h1>

- Merge test and browsing data 

In [4]:
browsing = pd.read_csv(DATA_PATH_BROWSING, sep=',')
# Add columns 'is_search' and 'is_test'
browsing['is_search'] = 0 
browsing['is_test'] = 0 
# Load test data 
with open('/workspace/intention_test_phase_2.json') as json_file:
    # read the test cases from the provided file
    test_queries = json.load(json_file)
# Add browsing events from test data
test_df = pd.json_normalize(test_queries, 'query', 'nb_after_add')
test_df['is_test'] = 1
print("There is %s unique sessions in Test table" %test_df.session_id_hash.nunique())
test_browsing = test_df[['session_id_hash', 'event_type', 'product_action', 'product_sku_hash',
       'server_timestamp_epoch_ms', 'hashed_url', 'is_search', 'is_test', 'nb_after_add']]
test_browsing = test_browsing[test_browsing.is_search==False]
# Concat train test browsing data to create event table 
event_df = pd.concat([browsing, test_browsing])
event_df.reset_index(drop=True, inplace=True)
del browsing

There is 47711 unique sessions in Test table


- Process duplicated events: which are defined as interactions that occur in the same session and at the same time 

In [5]:
tmp = event_df[(event_df.event_type == 'pageview') & (event_df.duplicated(['session_id_hash' , 'server_timestamp_epoch_ms']))]
event_df.drop(tmp.index, inplace=True)

- Keep the mapping of test sessions to their nb_after_add before merging with search table 

In [6]:
test_session_mapping= dict(zip(test_df.session_id_hash, test_df.nb_after_add))

- Generate search table from train and test data 

In [7]:
import ast
# helper function to convert string to list object
def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)

In [8]:
# Load search data
search = pd.read_csv("/workspace/search_train.csv", sep=',')
# Add column event_type 
search['event_type'] = 'search'
# Add column 'is_search'
search['is_search'] = 1
search['is_test'] = 0
# Drop 123 rows where: (clicked_skus_hash != NaN) and (product_skus_hash == NaN)
condition = (search['product_skus_hash'].isnull()) & (~search['clicked_skus_hash'].isnull())
search = search.loc[~condition]
# Convert strings to list object 
for col in ['product_skus_hash', 'clicked_skus_hash', 'query_vector']: 
    search[col] = search[col].progress_apply(convert_str_to_list)
# Add search events from test data
test_search = test_df[['session_id_hash', 'query_vector', 'clicked_skus_hash',
       'product_skus_hash', 'server_timestamp_epoch_ms', 'event_type',
       'is_search', 'is_test']]
test_search = test_search[test_search.is_search==True]
# Concat test and train search data
search = pd.concat([search, test_search])
search.reset_index(inplace=True)
# Compute number of returned and clicked items 
search['impression_size'] = search.product_skus_hash.str.len().fillna(0)
search['clicks_size'] = search.clicked_skus_hash.str.len().fillna(0)
# Compute number of search queries per session 
tmp = search.groupby('session_id_hash').size().reset_index()
tmp.columns = ['session_id_hash', 'nb_queries']
search = search.merge(tmp, on='session_id_hash', how='left')
# Update list of impressions by the clicked item when it is missing
def add_clicked(x): 
    if isinstance(x.clicked_skus_hash, list) and isinstance(x.product_skus_hash, list):
        return list(set(x.product_skus_hash).union(set(x.clicked_skus_hash)))
    return x.product_skus_hash
search['updated_product_skus_hash'] = search.progress_apply(add_clicked, axis=1)

100%|██████████| 834938/834938 [00:28<00:00, 29054.48it/s]


In [9]:
def add_clicked(x): 
    if isinstance(x.clicked_skus_hash, list) and isinstance(x.product_skus_hash, list):
        return list(set(x.product_skus_hash).union(set(x.clicked_skus_hash)))
    return x.product_skus_hash
search['updated_product_skus_hash'] = search.progress_apply(add_clicked, axis=1)

100%|██████████| 834938/834938 [00:29<00:00, 28200.19it/s]


- Define the session search as a sequence of search queries and the  interacted items 

In [10]:
def all_products(x): 
    t =[]
    for products in x.dropna(): 
        t += products
    if len(t)==0:
        return ['missing']
    return t

session_search = search.sort_values(['session_id_hash',
                                     'server_timestamp_epoch_ms']).groupby('session_id_hash').agg({'query_vector': lambda x: list(np.concatenate(x.values)),
                                                                                                                    'updated_product_skus_hash': all_products,
                                                                                                                    'clicked_skus_hash': all_products,
                                                                                                                    'impression_size': list,
                                                                                                                    'clicks_size': list,
                                                                                                                    'nb_queries': 'last'
                                                                                                                  })
session_search.columns = ['flat_query_vector', 'flat_product_skus_hash', 'flat_clicked_skus_hash', 'impressions_size', 'clicks_size', 'nb_queries']
session_search['clicked-flag'] = session_search.progress_apply(lambda x: [int(e in x['flat_clicked_skus_hash']) for e in x['flat_product_skus_hash']], axis=1)
session_search = session_search.reset_index()

100%|██████████| 560394/560394 [00:55<00:00, 10053.07it/s]


- Save search tables 

In [11]:
session_search.to_parquet(os.path.join(OUTPUT_DIR, "session_search.parquet"))
search.to_parquet(os.path.join(OUTPUT_DIR, "search.parquet"))

- Add search clicks as an additional product_action in the event_table 

In [12]:
# select search events with clicks
use_cols = ['session_id_hash', 'clicked_skus_hash',
            'server_timestamp_epoch_ms', 'event_type',
            'is_search']
search_clicks = search[search.clicks_size>0][use_cols]
print("There are %s search sessions that generate a click" %search_clicks.session_id_hash.nunique())
# Create new event-type and product-action
search_clicks['event_type'] = 'search'
search_clicks['product_action'] = 'click'
search_clicks['is_test'] = search_clicks['session_id_hash'].isin(test_df.session_id_hash.unique()).astype(int)
# Unstack the list of clicked items to multiple rows : each row is a single clicked item 
lst_col = 'clicked_skus_hash'
search_clicks = pd.DataFrame({
    col:np.repeat(search_clicks[col].values, search_clicks[lst_col].str.len()) for col in search_clicks.columns.difference([lst_col])}).assign(
    **{lst_col:np.concatenate(search_clicks[lst_col].values)})[search_clicks.columns.tolist()]
search_clicks.columns = ['session_id_hash', 'product_sku_hash', 'server_timestamp_epoch_ms',
                         'event_type', 'is_search', 'product_action', 'is_test']
# add nb_after_add for click events from the task mapping 
search_clicks['nb_after_add'] = search_clicks.session_id_hash.map(test_session_mapping)
# Add search clicks to event table 
event_df = pd.concat([event_df, search_clicks])
event_df.event_type.value_counts()

There are 157493 search sessions that generate a click


pageview         20967984
event_product    10671295
search             402589
Name: event_type, dtype: int64

- Fill missing product ids of pageview events with the url of the page : new column 'product_url_hash' is created

In [13]:
event_df['product_url_hash'] = event_df['product_sku_hash'].fillna(event_df['hashed_url'])

-  Keep sessions with at least one 'add' product_action 

In [14]:
sessions_with_add_mask = event_df.groupby('session_id_hash').progress_apply(lambda x: 'add' in x.product_action.values)
print(" \n There are %s sessions with at least one add-to-cart (AC) event" %sessions_with_add_mask.sum())
sessions_with_add = sessions_with_add_mask[sessions_with_add_mask == True].index.tolist()
event_df =event_df[event_df.session_id_hash.isin(sessions_with_add)]

100%|██████████| 4982436/4982436 [06:37<00:00, 12539.84it/s]


 
 There are 262395 sessions with at least one add-to-cart (AC) event


-  Add product information

In [15]:
product_info = pd.read_csv('/workspace/sku_to_content.csv')
def product_main_category(x):
    if pd.isna(x):
        return x
    return x.split('/')[0]

# Extract product main category
product_info['main_category'] = product_info['category_hash'].progress_apply(product_main_category)

# Compute average price of main and hierarchy category
main_price = product_info.groupby('main_category')['price_bucket'].mean().reset_index()
main_price.columns = ['main_category', 'mean_price_main']
hierarchy_price = product_info.groupby('category_hash')['price_bucket'].mean().reset_index()
hierarchy_price.columns = ['category_hash', 'mean_price_hierarchy']
product_info = product_info.merge(main_price, on=['main_category'], how='left')
product_info = product_info.merge(hierarchy_price, on=['category_hash'], how='left')

# Merge the event table with product information 
event_df = event_df.merge(product_info[['product_sku_hash', 'main_category', 'category_hash',
                                        'price_bucket', 'mean_price_hierarchy', 'mean_price_main' ]], 
                          on='product_sku_hash', how='left')

100%|██████████| 66386/66386 [00:00<00:00, 461480.56it/s]


- Create features related to the first purchase event within a session : 
             'first_purchase_id', 'first_purchase_position', 'is_purchased', 'purchase_timestamp'

In [16]:
def get_purchase_index(x): 
    if 'purchase' not in x.product_action.values: 
        return ['no_purchase', len(x), 0, x.server_timestamp_epoch_ms.values.max()]
    position =  x.product_action.values.tolist().index('purchase')
    purchase_id = x.product_sku_hash.values.tolist()[position]
    is_purchased = 1 
    purchase_timestamp = x.server_timestamp_epoch_ms.values.tolist()[position]
    return (purchase_id, position, is_purchased, purchase_timestamp)
purchase_event = event_df.groupby('session_id_hash').progress_apply(get_purchase_index)
purchase_event.columns = ['session_id_hash', 'purchase_features']
purchase_event = pd.DataFrame(purchase_event.tolist(), index= purchase_event.index).reset_index()
purchase_event.columns = ['session_id_hash', 'first_purchase_id',
                          'first_purchase_position', 'is_purchased',
                          'purchase_timestamp']
# merge purchase features and event table 
event_df = event_df.merge(purchase_event, on='session_id_hash', how='left')
print("There are %s purchase events" %event_df.drop_duplicates('session_id_hash').is_purchased.sum())

100%|██████████| 262395/262395 [00:45<00:00, 5738.59it/s]


There are 46138 purchase events


* Keep only the interactions that happened before the first purchase events 

In [17]:
event_df = event_df[event_df['server_timestamp_epoch_ms'] <= event_df['purchase_timestamp']]

- Drop sessions that lost the AC event after filtering out interactions happening after the first puchase

In [18]:
sessions_with_add_mask = event_df.groupby('session_id_hash').progress_apply(lambda x: 'add' in x.product_action.values)
print(" \n We keep %s sessions with at least one add-to-cart (AC) event before purchase" %sessions_with_add_mask.sum())
sessions_with_add = sessions_with_add_mask[sessions_with_add_mask == True].index.tolist()
event_df =event_df[event_df.session_id_hash.isin(sessions_with_add)]

100%|██████████| 262395/262395 [00:21<00:00, 11954.87it/s]


 
 We keep 262296 sessions with at least one add-to-cart (AC) event before purchase


- Create features related to the first AC event : 
            'AC_position', 'first_AC_id', 'original_nb_after_add'

In [19]:
def first_ac_product(x):
    add_index = x.product_action.values.tolist().index('add')
    product_id = x['product_sku_hash'].values.tolist()[add_index]
    nb_after = len(x[add_index+1:])
    return [product_id, nb_after]
add_event = event_df.sort_values(['session_id_hash',
                                  'server_timestamp_epoch_ms']).groupby('session_id_hash').progress_apply(first_ac_product)
add_event = pd.DataFrame(add_event.tolist(), index= add_event.index).reset_index()
add_event.columns = ['session_id_hash', 'first_AC_id', 'original_nb_after_add']
event_df = event_df.merge(add_event, on='session_id_hash', how='left')

100%|██████████| 262296/262296 [00:46<00:00, 5637.44it/s]


- Remove duplicated interactions of the same product / url within a session
 - All the interactions of the product within the session will be aggregated in boolean flags 
 - The number of interactions of each product is also computed 

In [20]:
# Aggregate product interactions 
same_interactions = event_df.groupby(['session_id_hash', 'product_url_hash']).agg({
    'product_url_hash': len,
    'product_action': [lambda x: 'add' in list(x), 
                       lambda x: 'detail' in list(x),
                       lambda x: 'remove' in list(x),
                       lambda x: 'view' in list(x),
                       lambda x: 'click' in list(x)
                      ]
}).droplevel(0, axis=1)
same_interactions = same_interactions.reset_index()
same_interactions.columns = ['session_id_hash', 'product_url_hash', 'product_nb_interactions',
                                'has_been_added_to_cart', 'has_been_detailed', 
                                 'has_been_removed_from_cart', 'has_been_viewed', 'has_been_clicked']
# drop duplicated events from event-table 
event_df = event_df.sort_values(['session_id_hash', 'server_timestamp_epoch_ms'])
event_df = event_df.drop_duplicates(['session_id_hash','product_url_hash'], keep='first')

# merge aggregated information with event-table
event_df = event_df.merge(same_interactions, on=['session_id_hash', 'product_url_hash'], how='left')

- The final event table is : 

In [21]:
display(event_df)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,is_test,nb_after_add,product_url_hash,main_category,category_hash,price_bucket,mean_price_hierarchy,mean_price_main,first_purchase_id,first_purchase_position,is_purchased,purchase_timestamp,first_AC_id,original_nb_after_add,product_nb_interactions,has_been_added_to_cart,has_been_detailed,has_been_removed_from_cart,has_been_viewed,has_been_clicked
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,0,NaN,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,NaN,NaN,NaN,NaN,NaN,no_purchase,16,0,1552426869735,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,13,1,False,False,False,False,False
1,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,detail,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,0,NaN,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.692630,no_purchase,16,0,1552426869735,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,13,3,True,True,False,False,False
2,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424395594,0ad6fab1eb3ac76010ea2fa6399a4e993b00f6501c88a2...,0,0,NaN,0ad6fab1eb3ac76010ea2fa6399a4e993b00f6501c88a2...,NaN,NaN,NaN,NaN,NaN,no_purchase,16,0,1552426869735,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,13,1,False,False,False,False,False
3,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424417587,e93e5c83aab0987e41d8fd65a30b54d2ce87491b4a7f9b...,0,0,NaN,e93e5c83aab0987e41d8fd65a30b54d2ce87491b4a7f9b...,NaN,NaN,NaN,NaN,NaN,no_purchase,16,0,1552426869735,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,13,1,False,False,False,False,False
4,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424698656,433b0e71df1fe9a8d1f45647545701f6108414c40eef76...,0,0,NaN,433b0e71df1fe9a8d1f45647545701f6108414c40eef76...,NaN,NaN,NaN,NaN,NaN,no_purchase,16,0,1552426869735,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,13,1,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065716,ffff42ffb7ff037c9677a014f450b173e8dc6bc455fdda...,pageview,NaN,NaN,1552496701137,0ad6fab1eb3ac76010ea2fa6399a4e993b00f6501c88a2...,0,0,NaN,0ad6fab1eb3ac76010ea2fa6399a4e993b00f6501c88a2...,NaN,NaN,NaN,NaN,NaN,no_purchase,42,0,1552497020769,8ea36b9be70e96438ffa1bbd7666f51da07bae997fc0be...,10,1,False,False,False,False,False
3065717,ffff42ffb7ff037c9677a014f450b173e8dc6bc455fdda...,pageview,NaN,NaN,1552496985547,c3f5e06de3008d158bf505f7e414975652c2b6c0a734e5...,0,0,NaN,c3f5e06de3008d158bf505f7e414975652c2b6c0a734e5...,NaN,NaN,NaN,NaN,NaN,no_purchase,42,0,1552497020769,8ea36b9be70e96438ffa1bbd7666f51da07bae997fc0be...,10,1,False,False,False,False,False
3065718,ffffcc02c4406d87ef142496eec68ccdfb97719f2cde92...,pageview,NaN,NaN,1551044465124,9c59aa6ff31e455fe5444bc40f1d6422c8403244ab9502...,0,0,NaN,9c59aa6ff31e455fe5444bc40f1d6422c8403244ab9502...,NaN,NaN,NaN,NaN,NaN,no_purchase,11,0,1551046833711,eae56abc8ed5f4921817bea41a7880df350cd4db252c26...,0,4,False,False,False,False,False
3065719,ffffcc02c4406d87ef142496eec68ccdfb97719f2cde92...,event_product,detail,eae56abc8ed5f4921817bea41a7880df350cd4db252c26...,1551044465124,9c59aa6ff31e455fe5444bc40f1d6422c8403244ab9502...,0,0,NaN,eae56abc8ed5f4921817bea41a7880df350cd4db252c26...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.692630,no_purchase,11,0,1551046833711,eae56abc8ed5f4921817bea41a7880df350cd4db252c26...,0,7,True,True,False,False,False


- Save to parquet file with 10 partitions 

In [22]:
event_df['parquet_split'] =  np.random.randint(0, 10, size=event_df.shape[0])
event_df.to_parquet(os.path.join(OUTPUT_DIR, "event_table"), partition_cols=['parquet_split'])

In [23]:
del event_df, same_interactions
gc.collect()

63

<h1> <center> <a id='row_workflow'> Define the preprocessed row interactions table </a> </center></h1>

In [3]:
files = glob.glob(OUTPUT_DIR + '/event_table/parquet_split*/*.parquet')

-  Workflow: Fill missing values, encode categorical variables and normalize numericals

In [4]:
#  load data 
df_event = nvt.Dataset(files, part_size="1GB") 

# convert timestamp to datetime object
to_datetime = ["server_timestamp_epoch_ms"] >> nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms')) >> nvt.ops.Rename( f = lambda x: 'timestamp')

# fill missing product ids before categorify to keep id '0' for padding 
missing_ids = ['product_sku_hash','hashed_url'] >> nvt.ops.FillMissing(fill_val='missing')
cat_product_ids = missing_ids >> nvt.ops.Categorify()

#joint encode product url pruchase_event and add_event ids
cat_joint_product_ids = [['product_url_hash', 'first_purchase_id', 'first_AC_id']] >> nvt.ops.Categorify()

# Encode the categorical features
categ_feats = ['session_id_hash', 'product_action',  'event_type', 'price_bucket', 'main_category', 'category_hash']
cat_feats = categ_feats >> nvt.ops.Categorify()

# Fill and normalize numerical features 
cont_feats = ['mean_price_hierarchy', 'mean_price_main'] >> nvt.ops.FillMedian()
continuous_feats = cont_feats >> nvt.ops.Normalize()

# Fill boolean flags 
interaction_flag = ['product_nb_interactions', 'has_been_added_to_cart',
                    'has_been_detailed', 'has_been_removed_from_cart',
                    'has_been_viewed', 'has_been_clicked']
interaction_flag =  interaction_flag >> nvt.ops.FillMissing(fill_val=0)


# Keep original value of the remaining features 
other_feats =   ['is_purchased', 'purchase_timestamp', 'original_nb_after_add']

# Define and fit the workflow
workflow = nvt.Workflow(['nb_after_add', 'is_search', 'is_test'] + cat_feats  + cat_product_ids + cat_joint_product_ids + \
                        interaction_flag + other_feats + to_datetime  + continuous_feats)
workflow.fit(df_event)

# Transform event table 
new_gdf = workflow.transform(df_event).to_ddf().compute()

# Include the item first time seen feature (for recency calculation) : Using product_url_hash column 
items_first_ts_df = new_gdf.groupby('product_url_hash').agg({'timestamp': 'min'}).reset_index().rename(columns={'timestamp': 'itemid_ts_first'})
interactions_merged_df = new_gdf.merge(items_first_ts_df, on=['product_url_hash'], how='left')

In [5]:
use_cols = ['session_id_hash', 'timestamp',  'event_type', 'product_action',
            
            'product_sku_hash', 'hashed_url', 'product_url_hash',
            
            'main_category', 'category_hash', 'price_bucket', 'mean_price_hierarchy', 'mean_price_main', 
            
            'itemid_ts_first', 'product_nb_interactions',
            
            'first_AC_id', 'has_been_added_to_cart', 'has_been_detailed', 'has_been_removed_from_cart', 'has_been_viewed', 'has_been_clicked',

            'first_purchase_id', 'is_purchased', 'purchase_timestamp',
            
            'nb_after_add', 'original_nb_after_add', 'is_search', 'is_test'] 

interactions_merged_df[use_cols].head(3)

,session_id_hash,timestamp,event_type,product_action,product_sku_hash,hashed_url,product_url_hash,main_category,category_hash,price_bucket,mean_price_hierarchy,mean_price_main,itemid_ts_first,product_nb_interactions,first_AC_id,has_been_added_to_cart,has_been_detailed,has_been_removed_from_cart,has_been_viewed,has_been_clicked,first_purchase_id,is_purchased,purchase_timestamp,nb_after_add,original_nb_after_add,is_search,is_test
0,15408,2019-03-05 18:55:37.157,2,0,55897,36834,39541,0,0,0,0.028997,0.039367,2019-01-15 05:12:25.905,5,44665,False,False,False,False,False,176371,0,1551813832328,<NA>,96,0,0
1,15408,2019-03-05 19:05:58.468,2,0,55897,65913,70522,0,0,0,0.028997,0.039367,2019-01-16 21:21:21.071,2,44665,False,False,False,False,False,176371,0,1551813832328,<NA>,96,0,0
2,15408,2019-03-05 19:22:26.174,2,0,55897,97493,104371,0,0,0,0.028997,0.039367,2019-01-16 04:03:06.466,1,44665,False,False,False,False,False,176371,0,1551813832328,<NA>,96,0,0


- Save resulting table and nvtabular workflow 

In [27]:
# save the workflow : 
workflow.save(os.path.join(OUTPUT_DIR, "categorify_workflow"))

# Save the parquet table 
interactions_merged_df[use_cols].to_parquet(os.path.join(OUTPUT_DIR , 'row_interactions_task2_preproc_v2.parquet'))

<h1> <center> <a id='session_workflow'>Preprocessing of session table  </a> </center></h1>


In [4]:
cont_feats = ['mean_price_hierarchy', 'mean_price_main', 'product_nb_interactions'] 
purchase_feat = ['first_AC_id', 'first_purchase_id', 'is_purchased', 'purchase_timestamp']
boolean_interactions = ['has_been_added_to_cart', 'has_been_detailed', 'has_been_removed_from_cart', 'has_been_viewed', 'has_been_clicked']
data_info = ['is_search', 'is_test'] 

In [5]:
interactions_merged_df = pd.read_parquet(os.path.join(OUTPUT_DIR , 'row_interactions_task2_preproc_v2.parquet'))

In [6]:
interactions_merged_df.head(3)

,session_id_hash,timestamp,event_type,product_action,product_sku_hash,hashed_url,product_url_hash,main_category,category_hash,price_bucket,mean_price_hierarchy,mean_price_main,itemid_ts_first,product_nb_interactions,first_AC_id,has_been_added_to_cart,has_been_detailed,has_been_removed_from_cart,has_been_viewed,has_been_clicked,first_purchase_id,is_purchased,purchase_timestamp,nb_after_add,original_nb_after_add,is_search,is_test
0,15408,2019-03-05 18:55:37.157,2,0,55897,36834,39541,0,0,0,0.028997,0.039367,2019-01-15 05:12:25.905,5,44665,False,False,False,False,False,176371,0,1551813832328,NaN,96,0,0
1,15408,2019-03-05 19:05:58.468,2,0,55897,65913,70522,0,0,0,0.028997,0.039367,2019-01-16 21:21:21.071,2,44665,False,False,False,False,False,176371,0,1551813832328,NaN,96,0,0
2,15408,2019-03-05 19:22:26.174,2,0,55897,97493,104371,0,0,0,0.028997,0.039367,2019-01-16 04:03:06.466,1,44665,False,False,False,False,False,176371,0,1551813832328,NaN,96,0,0


In [7]:
# convert booleans to int 
interactions_merged_df[boolean_interactions + data_info ] = interactions_merged_df[boolean_interactions + data_info ].astype('int32')

In [8]:
interactions_merged_df[interactions_merged_df.is_test==1].nb_after_add.isnull().sum()

0

In [9]:
# create time features
sessionTime = ['timestamp']

sessionTime_hour = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.hour) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.hour) >> 
    nvt.ops.Rename(postfix = '_hour')
)
sessionTime_weekday = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.weekday) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(postfix = '_wd')
)
sessionTime_day = (
    sessionTime >> 
    nvt.ops.LambdaOp(lambda col: col.dt.day) >> 
    nvt.ops.Rename(postfix="_day")
)

sessionTime_timestamp = (
    sessionTime >> 
    nvt.ops.LambdaOp(lambda col: (col.astype(int) / 1e6).astype(int)) >> 
    nvt.ops.Rename(f = lambda col: "ts")
)

# compute cycled features 
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

def get_cycled_feature_value_cos(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_cos = np.cos(2*np.pi*value_scaled)
    return value_cos
hour_sin = sessionTime_hour >> (lambda col: get_cycled_feature_value_sin(col, 24)) >> nvt.ops.Rename(postfix = '_sin')
hour_cos = sessionTime_hour >> (lambda col: get_cycled_feature_value_cos(col, 24)) >> nvt.ops.Rename(postfix = '_cos')
weekday_sin = sessionTime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(postfix = '_sin')
weekday_cos= sessionTime_weekday >> (lambda col: get_cycled_feature_value_cos(col+1, 7)) >> nvt.ops.Rename(postfix = '_cos')
cycled_features = hour_sin + hour_cos + weekday_sin + weekday_cos


# calculate item recency 
from nvtabular.ops import Operator
class ItemRecency(Operator):
    def transform(self, columns, gdf):
        for column in columns:
            col = gdf[column]
            #col.loc[col == ""] = None
            item_first_timestamp = gdf['itemid_ts_first']
            delta_days = (col - item_first_timestamp).dt.days
            gdf[column + "_age_days"] = delta_days * (delta_days >=0)
        return gdf
            
    def output_column_names(self, columns):
        return [column + "_age_days" for column in columns]
            
    def dependencies(self):
        return ["itemid_ts_first"]
recency_features = ["timestamp"] >> ItemRecency() 
recency_features_norm = recency_features >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(postfix = '_norm')

time_features = (
    sessionTime_timestamp +
    sessionTime + 
    sessionTime_hour +
    sessionTime_day + 
    sessionTime_weekday +
    recency_features +
    recency_features_norm + 
    cycled_features)

In [10]:
time_features.columns

['ts',
 'timestamp',
 'timestamp_hour',
 'timestamp_day',
 'timestamp_wd',
 'timestamp_age_days',
 'timestamp_age_days_norm',
 'timestamp_hour_sin',
 'timestamp_hour_cos',
 'timestamp_wd_sin',
 'timestamp_wd_cos']

- Grouping interactions into sessions

In [11]:
# Define Groupby Workflow: search columns are not used
# N.B: Add the op ListSlice when upgrading nvt 0.5.1 to 0.6 
filter_nan_products = (interactions_merged_df.columns >> nvt.ops.Filter(f=lambda df: df['product_sku_hash'] != 0))

groupby_only_product = filter_nan_products - ['timestamp']  + time_features  >> nvt.ops.Groupby(
    groupby_cols=["session_id_hash"], 
    sort_cols=["ts"],
    aggs={
       "product_sku_hash": ["list", "count"], 
    }
)
    
groupby_product_url = ['session_id_hash', 'product_url_hash']  + time_features >> nvt.ops.Groupby(
    groupby_cols=["session_id_hash"], 
    sort_cols=["ts"],
    aggs={
       "product_url_hash": ["list", "count"]
    }
)


groupby_other_features = ['session_id_hash', 'product_action', 'event_type', 'first_AC_id', 'original_nb_after_add',
                          'price_bucket', 'category_hash', 'main_category', 'nb_after_add' ] + data_info + boolean_interactions + purchase_feat + cont_feats + time_features >> \
    nvt.ops.Groupby(
    groupby_cols=["session_id_hash"], 
    sort_cols=["ts"],
    aggs={
        "product_action": ["list"],     
        "event_type": ["list"],    
        "price_bucket": ["list"],
        'main_category': ["list"],
        "category_hash": ["list"],
        'mean_price_hierarchy':["list"],
        'mean_price_main':["list"],
        "ts": ["list", "first", "last"],
        "is_test": ["last"],
        "is_search": ["last"],
        'nb_after_add': ["last"],
        "original_nb_after_add": ["last"],
        
        'first_AC_id': ["last"],
        'is_purchased': ["last"],
        'purchase_timestamp':["last"],

        'product_nb_interactions': ["list"],
        'has_been_added_to_cart': ["list"],
        'has_been_detailed': ["list"],
        'has_been_removed_from_cart': ["list"],
        'has_been_viewed': ["list"],
        'has_been_clicked': ["list"],

        "timestamp": ["first"],
        'timestamp_day': ["list"],
        'timestamp_hour': ["list"],
        'timestamp_month': ["list"],
        'timestamp_wd': ["list"],
        'timestamp_age_days': ["list"],
        'timestamp_age_days_norm': ["list"],
        'timestamp_hour_sin': ["list"],
        'timestamp_hour_sin_norm': ["list"],
        'timestamp_hour_cos': ["list"],
        'timestamp_hour_cos_norm': ["list"],
        'timestamp_wd_sin': ["list"],
        'timestamp_wd_sin_norm': ["list"],
        'timestamp_wd_cos': ["list"],
        'timestamp_wd_cos_norm': ["list"],   
        
        },
    name_sep="-")

- workflow 1 : group other features that user interactions 

In [12]:
remaining_columns = [x for x in groupby_other_features.columns if x!= 'timestamp-first']
day_index = ((groupby_other_features - remaining_columns)  >> 
    nvt.ops.LambdaOp(lambda col: (col.max() - col).dt.days + 1) >> 
    nvt.ops.Rename(f = lambda col: "day_index")
)              

In [13]:
workflow = nvt.Workflow(groupby_other_features + day_index)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
new_gdf_other = workflow.transform(dataset).to_ddf().compute()
len(new_gdf_other)

262296

In [14]:
new_gdf_other.head(2)

,ts-last,timestamp_wd_cos-list,timestamp_hour_sin-list,session_id_hash,mean_price_main-list,is_purchased-last,ts-first,nb_after_add-last,timestamp_wd_sin-list,has_been_removed_from_cart-list,timestamp_age_days-list,original_nb_after_add-last,timestamp_wd-list,has_been_viewed-list,timestamp_hour-list,product_nb_interactions-list,mean_price_hierarchy-list,ts-list,is_test-last,category_hash-list,product_action-list,has_been_added_to_cart-list,event_type-list,price_bucket-list,has_been_detailed-list,purchase_timestamp-last,is_search-last,has_been_clicked-list,first_AC_id-last,timestamp_age_days_norm-list,timestamp-first,timestamp_day-list,main_category-list,timestamp_hour_cos-list,day_index
0,1552426867019,"[-0.22252177, -0.22252177, -0.22252177, -0.222...","[-0.866025, -0.866025, -0.866025, -0.70710653,...",1,"[0.039366912, 0.039366912, 0.039366912, 0.0393...",0,1552423391039,<NA>,"[0.9749277, 0.9749277, 0.9749277, 0.9749277, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[52, 52, 56, 56, 56, 56, 56, 56, 42, 49, 0, 0,...",13,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.028996615, 2.661621, 0.028996615, 0.0289966...","[1552423391039, 1552423391039, 1552424395593, ...",0,"[0, 86, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 86, 0]","[0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2]","[0, 10, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 10, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]",1552426869735,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",143199,"[0.6673608, 0.6673608, 0.72311467, 0.72311467,...",2019-03-12 20:43:11.039,"[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...","[0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0]","[0.5000007, 0.5000007, 0.5000007, 0.707107, 0....",63
1,1557247660313,"[-0.22252177, -0.22252177]","[-0.8660257, -0.8660257]",2,"[0.039366912, 0.039366912]",0,1557247655055,0.0,"[0.9749277, 0.9749277]","[0, 0]","[112, 111]",0,"[1, 1]","[0, 0]","[16, 16]","[1, 2]","[1.17335, 1.17335]","[1557247655055, 1557247660313]",1,"[60, 60]","[3, 3]","[0, 1]","[1, 1]","[8, 8]","[1, 1]",1557247744887,0,"[0, 0]",77197,"[1.2475067, 1.2406951]",2019-05-07 16:47:35.055,"[7, 7]","[2, 2]","[-0.4999995, -0.4999995]",7


- workflow 2 : create the sequence of product interactions and pageviews

In [15]:
workflow = nvt.Workflow(groupby_product_url)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
new_gdf_sku_url = workflow.transform(dataset).to_ddf().compute()
len(new_gdf_sku_url)

262296

In [16]:
new_gdf_sku_url.head()

,session_id_hash,product_url_hash_list,product_url_hash_count
0,1,"[7298, 143199, 7449, 160654, 46772, 11561, 130...",14
1,2,"[130669, 77197]",2
2,3,"[53641, 91938, 165669, 106467, 133979, 156658,...",32
3,4,"[170620, 107009, 153920]",3
4,5,"[46772, 121609, 63033, 123318, 122002, 801, 97...",59


- workflow 3 : create sequence with only product interactions

In [17]:
workflow = nvt.Workflow(groupby_only_product)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
new_gdf_prod_only = workflow.transform(dataset).to_ddf().compute()
len(new_gdf_prod_only)

262296

- Merge the three resulting frames

In [18]:
sessions_gdf = new_gdf_sku_url.merge(new_gdf_other, on='session_id_hash',  how='inner')
sessions_gdf = sessions_gdf.merge(new_gdf_prod_only,  on='session_id_hash',  how='left' )

In [19]:
sessions_gdf.shape

(262296, 39)

In [20]:
sessions_gdf.columns

Index(['session_id_hash', 'product_url_hash_list', 'product_url_hash_count',
       'ts-last', 'timestamp_wd_cos-list', 'timestamp_hour_sin-list',
       'mean_price_main-list', 'is_purchased-last', 'ts-first',
       'nb_after_add-last', 'timestamp_wd_sin-list',
       'has_been_removed_from_cart-list', 'timestamp_age_days-list',
       'original_nb_after_add-last', 'timestamp_wd-list',
       'has_been_viewed-list', 'timestamp_hour-list',
       'product_nb_interactions-list', 'mean_price_hierarchy-list', 'ts-list',
       'is_test-last', 'category_hash-list', 'product_action-list',
       'has_been_added_to_cart-list', 'event_type-list', 'price_bucket-list',
       'has_been_detailed-list', 'purchase_timestamp-last', 'is_search-last',
       'has_been_clicked-list', 'first_AC_id-last',
       'timestamp_age_days_norm-list', 'timestamp-first', 'timestamp_day-list',
       'main_category-list', 'timestamp_hour_cos-list', 'day_index',
       'product_sku_hash_list', 'product_sku_hash_c

- Display resulting session table 

In [45]:
SELECTED_COLS = ['session_id_hash', 'day_index', 'product_url_hash_list',
                 'event_type-list', 'product_action-list', 
                
                'category_hash-list', 'main_category-list',
                'price_bucket-list', 'mean_price_hierarchy-list', 'mean_price_main-list',
                
                'product_nb_interactions-list', 'has_been_added_to_cart-list', 'has_been_detailed-list', 'has_been_removed_from_cart-list',
                'has_been_viewed-list',  'has_been_clicked-list',
                
                'product_sku_hash_count',  'product_sku_hash_list',
                'product_url_hash_count',
                
                 'ts-first', 'ts-last',  'ts-list',
                 'timestamp_hour_cos-list', 'timestamp_hour_sin-list', 'timestamp_wd_sin-list', 'timestamp_wd_cos-list',
                 'timestamp_age_days-list', 'timestamp_age_days_norm-list', 
                
                 'first_AC_id-last', 'is_purchased-last', 'purchase_timestamp-last', 'nb_after_add-last',
                 'is_search-last' ,'is_test-last',
                ] 
         

sessions_gdf = sessions_gdf[SELECTED_COLS]
sessions_gdf.tail(2)

,session_id_hash,day_index,product_url_hash_list,event_type-list,product_action-list,category_hash-list,main_category-list,price_bucket-list,mean_price_hierarchy-list,mean_price_main-list,product_nb_interactions-list,has_been_added_to_cart-list,has_been_detailed-list,has_been_removed_from_cart-list,has_been_viewed-list,has_been_clicked-list,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,ts-first,ts-last,ts-list,timestamp_hour_cos-list,timestamp_hour_sin-list,timestamp_wd_sin-list,timestamp_wd_cos-list,timestamp_age_days-list,timestamp_age_days_norm-list,first_AC_id-last,is_purchased-last,purchase_timestamp-last,nb_after_add-last,is_search-last,is_test-last
262294,260295,109,"[46772, 111985, 45686, 156137, 16323, 7449, 16...","[2, 3, 2, 2, 1, 2, 2, 2, 1]","[0, 2, 0, 0, 5, 0, 0, 0, 4]","[0, 107, 0, 0, 0, 0, 0, 0, 0]","[0, 2, 0, 0, 0, 0, 0, 0, 0]","[0, 8, 0, 0, 0, 0, 0, 0, 0]","[0.028996615, -0.12167655, 0.028996615, 0.0289...","[0.039366912, 0.039366912, 0.039366912, 0.0393...","[1, 7, 1, 1, 9, 1, 1, 1, 1]","[0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0]",9,"[55897, 35528, 55897, 55897, 5178, 55897, 5589...",9,1548458502000,1548458695729,"[1548458502000, 1548458512739, 1548458514245, ...","[0.965926, 0.965926, 0.965926, 0.965926, 0.965...","[-0.25881836, -0.25881836, -0.25881836, -0.258...","[-0.9749281, -0.9749281, -0.9749281, -0.974928...","[-0.22252008, -0.22252008, -0.22252008, -0.222...","[10, 9, 10, 9, 10, 10, 10, 10, 10]","[-0.5375318, -0.6105659, -0.5375318, -0.610565...",111985,1,1548458695729,<NA>,0,0
262295,260296,20,"[138514, 100990, 133584]","[1, 1, 1]","[3, 3, 3]","[87, 87, 87]","[2, 2, 2]","[10, 9, 9]","[1.7068545, 1.7068545, 1.7068545]","[0.039366912, 0.039366912, 0.039366912]","[1, 3, 1]","[0, 1, 0]","[1, 1, 1]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",3,"[43880, 31985, 42297]",3,1556122996489,1556123028428,"[1556122996489, 1556123020886, 1556123028428]","[-0.4999995, -0.4999995, -0.4999995]","[-0.8660257, -0.8660257, -0.8660257]","[0.43388295, 0.43388295, 0.43388295]","[-0.90096927, -0.90096927, -0.90096927]","[97, 97, 99]","[1.1383731, 1.1383731, 1.153854]",100990,0,1556123060835,0.0,0,1


- Un-hash session id 

In [52]:
session_map = cudf.read_parquet(OUTPUT_DIR + '/categorify_workflow/categories/unique.session_id_hash.parquet').reset_index()
session_map.columns = ['session_id_hash', 'original_session_id_hash']
sessions_gdf = sessions_gdf.merge(session_map, on=['session_id_hash'], how='left')

- Get index of AC product in the session 

In [56]:
sessions_gdf = sessions_gdf.to_pandas()

In [57]:
sessions_gdf['first_add_index'] = sessions_gdf.progress_apply(lambda x: x['product_url_hash_list'].tolist().index(x['first_AC_id-last']), axis=1)

100%|██████████| 262296/262296 [00:04<00:00, 53250.30it/s]


- Compute actual number of non repeated interactions after add 

In [62]:
def get_nb_after_add(x): 
    add_index =  x['first_add_index']
    nb_after_last_add = len(x['product_url_hash_list']) - add_index - 1
    return nb_after_last_add
sessions_gdf['original_nb_after_add'] = sessions_gdf.progress_apply(get_nb_after_add, axis=1)

100%|██████████| 262296/262296 [00:04<00:00, 59150.39it/s]


- Compute artificially designed `nb_after_add` for train and validation session : By selecting nb_after_add in (0, 2, 4, 6, 8, 10) closest to the given value of actual number of events after the first AC

In [65]:
NB_AFTER_ADD = [0, 2, 4, 6, 8, 10]
sessions_gdf.loc[sessions_gdf['is_test-last']==0, 'nb_after_add-last'] = \
sessions_gdf.loc[sessions_gdf['is_test-last']==0, 'original_nb_after_add'].progress_apply(
    lambda z: min(NB_AFTER_ADD, key=lambda x:abs(x-z)))

100%|██████████| 214585/214585 [00:00<00:00, 305131.99it/s]


- Create cross-validation folds : 
     - Define random 5 folds column 
     - Reserve the 3 last weeks for validation 

from sklearn.model_selection import GroupKFold
sessions_gdf['fold'] = np.random.randint(1,6, sessions_gdf.shape[0]) 
sessions_gdf['is_valid'] = 0 
sessions_gdf.loc[((sessions_gdf['is_test-last']==0) & (sessions_gdf['day_index']<=50)), 'is_valid'] = 1

### Save session table 

- Save the whole session table 

In [70]:
sessions_gdf.to_parquet(os.path.join(OUTPUT_DIR, 'session_interactions_task2_preproc2.parquet'))

-  Save unique product sku mapping from updated product_url_hash encoded column

In [73]:
urls_ids = interactions_merged_df[interactions_merged_df.event_type==2]['product_url_hash'].unique()
mapping = pd.read_parquet(OUTPUT_DIR + '/categorify_workflow/categories/unique.product_url_hash_first_purchase_id_first_AC_id.parquet')
mask = mapping.reset_index()['index'].isin(urls_ids)
mapping_prod = mapping[~mask].reset_index()
mapping_prod.columns =  ['encoded_product_sku', 'original_product_sku']
print("There are %s unique product" %mapping_prod.shape[0])
mapping_prod.to_parquet(OUTPUT_DIR +'/mapping_product_sku_without_urls_task2_v2.parquet')

There are 55898 unique product


- Save products embedding matrices based on their encoded ids 

In [74]:
product_info = pd.read_csv('/workspace/sku_to_content.csv', usecols=['product_sku_hash', 
                                                                     'description_vector', 
                                                                     'image_vector'])
# convert strings to list object 
import ast
def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)
for col in ['description_vector', 'image_vector']: 
    product_info[col] = product_info[col].progress_apply(convert_str_to_list)
product_info.columns = ['original_product_sku', 'description_vector', 'image_vector']

### Merge product embeddings and mapping_prod
embeddings_table = mapping_prod.merge(product_info, on=['original_product_sku'], how='left')

# Fill missing embeddings with vector of zeros 
embeddings_table.loc[embeddings_table.description_vector.isnull(),
                         'description_vector'] = pd.Series([np.zeros(50)] * embeddings_table.description_vector.isnull().sum()).values

embeddings_table.loc[embeddings_table.image_vector.isnull(),
                         'image_vector'] = pd.Series([np.zeros(50)] * embeddings_table.image_vector.isnull().sum()).values

# Create Numpy matrix with the image vectors of the products
image_matrix = np.concatenate(embeddings_table.image_vector.values).reshape(-1, 50)
# Create Numpy matrix with the description vectors of the products
desc_matrix = np.concatenate(embeddings_table.description_vector.values).reshape(-1, 50)
# Define a dictionary to map the encoded product_sku to the position in the embedding matrices
mapping_id_sku_emb_position = dict(zip(embeddings_table.encoded_product_sku, embeddings_table.index))
# Saving the objects:
import pickle
with open(OUTPUT_DIR+'/embedding_data_v2.pkl', 'wb') as f:  
    pickle.dump([desc_matrix, image_matrix, mapping_id_sku_emb_position], f)

100%|██████████| 66386/66386 [00:05<00:00, 13124.86it/s]


<h2> <center> <a id='session_duplicate'>  Duplicated train sessions with different split points after the AC event </a></center></h2>

- Load data 

In [4]:
DATA_FOLDER = '/workspace/coveo_task2_v1_phase2/sessions_wo_repetitions/'
# load session browsing data 
data = pd.read_parquet(os.path.join(DATA_FOLDER, "session_interactions_task2_preproc2.parquet"))
#load product embeddings 
desc_matrix, image_matrix, mapping_id_sku_emb_position = pickle.load(open(DATA_FOLDER + "/embedding_data_v2.pkl", "rb"))
#load encoded product-ids
mapping = pd.read_parquet(os.path.join(DATA_FOLDER,
                                       "categorify_workflow/categories/unique.product_url_hash_first_purchase_id_first_AC_id.parquet"))
#load session search data 
search_session = pd.read_parquet("/workspace/coveo_task2_v1_phase2/sessions_wo_repetitions/session_search.parquet")
search_session.columns = ['original_session_id_hash', 'flat_query_vector', 'flat_product_skus_hash',
       'flat_clicked_skus_hash', 'impressions_size', 'clicks_size',
       'nb_queries', 'clicked-flag']

- Repeat rows of sessions with different number of actions after the AC event : Only for train and validation datasets

In [6]:
def truncate_at_different_position(x, feature_list): 
    product_id =  x['first_AC_id-last']
    nb_after_add = x['nb_after_add-last']
    add_index = x['product_url_hash_list'].tolist().index(product_id)
    general_feat = []
    if nb_after_add == 0: 
        general_feat.append([x[col][:int(add_index+nb_after_add+1)] for col in feature_list])
    
    else:
        for i in range(0, int(nb_after_add)+2, 2) : 
            general_feat.append([x[col][0:int(add_index+i+1)] for col in feature_list])
    return general_feat

feature_list = [col for col in data.columns if 'list' in col]
non_list_features = [col for col in data.columns if 'list' not in col]

list_frame = data[data['is_test-last'] == 0][non_list_features].copy()
list_frame['dynamic_truncated_lists'] = data[data['is_test-last'] == 0].progress_apply(partial(truncate_at_different_position,
                                                                                               feature_list=feature_list), axis=1)
# Unstack the list of clicked items to multiple rows : each row is a single clicked item 
lst_col = 'dynamic_truncated_lists'
duplicated_sessions = pd.DataFrame({
    col:np.repeat(list_frame[col].values,
                  list_frame[lst_col].str.len()) for col in list_frame.columns.difference([lst_col])}).assign(
    **{lst_col:[item for sublist in list_frame[lst_col].values for item in sublist]})[list_frame.columns.tolist()]
# Unstack truncated lists to 22 feature)list columns 
t = pd.DataFrame(duplicated_sessions[lst_col].to_list(), columns=feature_list)
duplicated_frame = pd.concat([duplicated_sessions, t], axis=1)
duplicated_frame.drop(lst_col, axis=1, inplace=True)

100%|██████████| 214585/214585 [01:33<00:00, 2305.98it/s]


- Update nb_after_add-last column 

In [7]:
def get_nb_after_add(x): 
    product_id =  x['first_AC_id-last']
    add_index = x['product_url_hash_list'].tolist().index(product_id)
    nb_after_last_add = len(x['product_url_hash_list']) - add_index - 1
    return nb_after_last_add
duplicated_frame['updated_original_nb_after_add'] = duplicated_frame.progress_apply(get_nb_after_add, axis=1)

100%|██████████| 744493/744493 [00:19<00:00, 38491.34it/s]


In [8]:
NB_AFTER_ADD = [0, 2, 4, 6, 8, 10]
duplicated_frame.loc[duplicated_frame['is_test-last']==0, 'nb_after_add-last'] = \
duplicated_frame.loc[duplicated_frame['is_test-last']==0, 'updated_original_nb_after_add'].progress_apply(
    lambda z: min(NB_AFTER_ADD, key=lambda x:abs(x-z)))

100%|██████████| 744493/744493 [00:02<00:00, 337990.31it/s]


- Merge back with test sessions 

In [9]:
duplicated_frame = pd.concat([data[data['is_test-last']==1],duplicated_frame])

- Save dataset 

In [10]:
duplicated_frame.to_parquet(os.path.join(DATA_FOLDER, 'duplicated_sessions_with_different_nb_after_add_cuts.parquet'))